Window size - number of  of past values we are using  to predict  the  future value

<h3>Loading the libraries <h3>

In [2]:
import tensorflow as tf

In [27]:
dataset =tf.data.Dataset.range(10)  
for val in dataset:
    print(val)  
    # type of this val is tensor object, in our case it's scalar
    print(val.numpy())
    # turning tensor object into numpy array







tf.Tensor(0, shape=(), dtype=int64)
0
tf.Tensor(1, shape=(), dtype=int64)
1
tf.Tensor(2, shape=(), dtype=int64)
2
tf.Tensor(3, shape=(), dtype=int64)
3
tf.Tensor(4, shape=(), dtype=int64)
4
tf.Tensor(5, shape=(), dtype=int64)
5
tf.Tensor(6, shape=(), dtype=int64)
6
tf.Tensor(7, shape=(), dtype=int64)
7
tf.Tensor(8, shape=(), dtype=int64)
8
tf.Tensor(9, shape=(), dtype=int64)
9


2024-05-18 10:53:11.395969: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'\ndataset =  dataset.window(5,shift=1,drop_remainder=True)\ndataset =  dataset.flat_map(lambda window:window.batch(5))\nfor window in dataset:\n  print(window.numpy())\n'

In [ ]:
'''
dataset =  dataset.window(5,shift=1,drop_remainder=True)
dataset =  dataset.flat_map(lambda window:window.batch(5))
for window in dataset:
  print(window.numpy())
'''